# Álfröðull Cloud input generator

In [ ]:
import h5py

import pathlib
import re

import h5py
import imageio
import IPython.display as disp

import math
import numpy as np
import scipy

import pandas as pd

from scipy.interpolate import interp1d
from scipy.integrate import simps, quad

from astropy import constants as const
from astropy.modeling import models

from astropy import units as u

#%matplotlib ipympl
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# cloud input file from daniel

cloud_input = pathlib.Path("../input/cross_sections_mgsio3_r1mu.dat")
opacity_sample = pathlib.Path( "../input/opac_sample_r5.h5" )
cloud_sample_output = pathlib.Path( "../input/cloud_sample_r5.h5" )

# Load opacity wavelength bins

In [ ]:
# get wavelength bin centers
with h5py.File(opacity_sample, "r") as opac_h5:    
                        
    # wavelength grid                
    try:
        opac_wave = [x for x in opac_h5["center wavelengths"][:]]
    except KeyError:
        opac_wave = [x for x in opac_h5["wavelengths"][:]]
        nbin = len(opac_wave)
                    
                                       
    # interface positions of the wavelength bins
    try:
        opac_interwave = [i for i in opac_h5["interface wavelengths"][:]]
    except KeyError:
        # quick and dirty way to get the lamda interface values
        opac_interwave = []
        opac_interwave.append(opac_wave[0] - (opac_wave[1] - opac_wave[0])/2)
        for x in range(len(opac_wave) - 1):
            opac_interwave.append((opac_wave[x+1] + opac_wave[x])/2)
        opac_interwave.append(opac_wave[-1] + (opac_wave[-1] - opac_wave[-2])/2)

        # widths of the wavelength bins
        try:
            opac_deltawave = [w for w in opac_h5["wavelength width of bins"][:]]
        except KeyError:
            opac_deltawave = []
            for x in range(len(opac_interwave) - 1):
                opac_deltawave.append(opac_interwave[x + 1] - opac_interwave[x])
        


In [ ]:
# convert units :
opac_bin_centers_wavelengths = np.array(opac_wave)*1e-2
opac_bin_interfaces_wavelengths = np.array(opac_interwave)*1e-2

print(f"Found {len(opac_bin_centers_wavelengths)} opacity wavelength bins")
print(f"from {opac_bin_interfaces_wavelengths[0]} m to {opac_bin_interfaces_wavelengths[-1]} m ")

# Load cloud data

In [ ]:

#wavelengths (mu)
#size parameter  
#extinction cross section (cm^2)  
#scattering cross section (cm^2)  
#absorption cross section (cm^2)  
#single scattering albedo         
#asymmetry parameter  

names = [
"wavelengths",
"size_parameter",
"extinction_cross_section",
"scattering_cross_section",
"absorption_cross_section",
"single_scattering_albedo",
"asymmetry_parameter" ]

cloud_input_data = pd.read_csv(cloud_input, sep='\s+', header=None, skiprows=1, names=names)


In [ ]:
# visual check of the content
cloud_input_data

### load and convert to SI

In [ ]:
cloud_wavelength = cloud_input_data['wavelengths'].values*1e-6
cloud_absorption = cloud_input_data['absorption_cross_section'].values*1e-4
cloud_scattering = cloud_input_data["scattering_cross_section"].values*1e-4

cloud_asymmetry = cloud_input_data['asymmetry_parameter'].values

print(f"Found {len(cloud_wavelength)} bins")
print(f"from {cloud_wavelength[0]} m to {cloud_wavelength[-1]} m")

# interpolate to opacity bin centers


In [ ]:
interpkind = 'cubic'

cloud_absorption_out = interp1d(cloud_wavelength, 
                                 cloud_absorption, 
                                 bounds_error=False,
                                 kind=interpkind,
                                 fill_value='extrapolate' )(opac_bin_centers_wavelengths)

cloud_absorption_log_out = np.exp(interp1d(cloud_wavelength, 
                                 np.log(cloud_absorption), 
                                 bounds_error=False,
                                 kind=interpkind,
                                 fill_value='extrapolate' )(opac_bin_centers_wavelengths))

cloud_absorption_loglog_out = np.exp(interp1d(np.log(cloud_wavelength), 
                                 np.log(cloud_absorption), 
                                 bounds_error=False,
                                 kind=interpkind,
                                 fill_value='extrapolate' )(np.log(opac_bin_centers_wavelengths)))

cloud_absorption_loglin_out = interp1d(np.log(cloud_wavelength), 
                                 cloud_absorption, 
                                 bounds_error=False,
                                 kind=interpkind,
                                 fill_value='extrapolate' )(np.log(opac_bin_centers_wavelengths))

In [ ]:
cloud_scattering_out = interp1d(cloud_wavelength, 
                                 cloud_scattering, 
                                 bounds_error=False,
                                 kind=interpkind,
                                 fill_value='extrapolate' )(opac_bin_centers_wavelengths)

cloud_scattering_log_out = np.exp(interp1d(cloud_wavelength, 
                                 np.log(cloud_scattering), 
                                 bounds_error=False,
                                 kind=interpkind,
                                 fill_value='extrapolate' )(opac_bin_centers_wavelengths))
                                     
cloud_scattering_loglog_out = np.exp(interp1d(np.log(cloud_wavelength), 
                                 np.log(cloud_scattering), 
                                 bounds_error=False,
                                 kind=interpkind,
                                 fill_value='extrapolate' )(np.log(opac_bin_centers_wavelengths)))

cloud_scattering_loglin_out = interp1d(np.log(cloud_wavelength), 
                                 cloud_scattering, 
                                 bounds_error=False,
                                 kind=interpkind,
                                 fill_value='extrapolate' )(np.log(opac_bin_centers_wavelengths))

In [ ]:
cloud_asymmetry_out = interp1d(cloud_wavelength, 
                                 cloud_asymmetry, 
                                 bounds_error=False,
                                 kind=interpkind,
                                 fill_value='extrapolate' )(opac_bin_centers_wavelengths)

cloud_asymmetry_loglin_out = interp1d(np.log(cloud_wavelength), 
                                 cloud_asymmetry, 
                                 bounds_error=False,
                                 kind=interpkind,
                                 fill_value='extrapolate' )(np.log(opac_bin_centers_wavelengths))

cloud_asymmetry_loglog_out = np.exp(interp1d(np.log(cloud_wavelength), 
                                 np.log(cloud_asymmetry), 
                                 bounds_error=False,
                                 kind=interpkind,
                                 fill_value='extrapolate' )(np.log(opac_bin_centers_wavelengths)))

# plots

#### check of bin repartition

In [ ]:
fig, ax_lambda = plt.subplots(1,1, figsize=(14,10))

ax_lambda.plot(cloud_wavelength, marker='+', linestyle='None', label='cloud input')
ax_lambda.plot(opac_bin_centers_wavelengths, marker='+', linestyle='None', label='opacity input')

ax_lambda.set_yscale('log')

ax_lambda.set_xlabel('index')
ax_lambda.set_ylabel('Wavelength [m]')

ax_lambda.legend();

### Absorption

In [ ]:
def plt_func(ax, x_axes, y_axes, labels, xlabel, ylabel, markers, xscale, yscale):
    
    for x, y, l, m in zip(x_axes, y_axes, labels, markers):
        ax.plot(x, y, marker=m, label=l)

    ax.set_xscale(xscale)
    ax.set_yscale(yscale)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.legend()

In [ ]:
fig, (ax_g0_lin, ax_g0_log) = plt.subplots(1,2, figsize=(2*14,10))


plt_func(ax_g0_lin, 
         [cloud_wavelength,opac_bin_centers_wavelengths,opac_bin_centers_wavelengths,opac_bin_centers_wavelengths,opac_bin_centers_wavelengths,opac_bin_centers_wavelengths],
         [cloud_absorption, cloud_absorption_out, cloud_absorption_log_out, cloud_absorption_loglog_out, cloud_absorption_loglin_out],
         ['orig', 'interp', 'interp_log', 'interp_loglog', 'interp_loglin'],
         "Wavelength [m]",
         "absorption cross section [$m^{-2}$]",
         ['+', 'x', '1', '2', '3'],
         'log',
         'linear' )

plt_func(ax_g0_log, 
         [cloud_wavelength,opac_bin_centers_wavelengths,opac_bin_centers_wavelengths,opac_bin_centers_wavelengths,opac_bin_centers_wavelengths,opac_bin_centers_wavelengths],
         [cloud_absorption, cloud_absorption_out, cloud_absorption_log_out, cloud_absorption_loglog_out, cloud_absorption_loglin_out],
         ['orig', 'interp', 'interp_log', 'interp_loglog', 'interp_loglin'],
         "Wavelength [m]",
         "absorption cross section [$m^{-2}$]",
         ['+', 'x', '1', '2', '3'],
         'log',
         'log' )

pass

### Scattering

In [ ]:
fig, (ax_sigma_scat_lin, ax_sigma_scat_log) = plt.subplots(1,2, figsize=(2*14,10))


plt_func(ax_sigma_scat_lin, 
         [cloud_wavelength,opac_bin_centers_wavelengths,opac_bin_centers_wavelengths,opac_bin_centers_wavelengths,opac_bin_centers_wavelengths,opac_bin_centers_wavelengths],
         [cloud_scattering, cloud_scattering_out, cloud_scattering_log_out, cloud_scattering_loglog_out, cloud_scattering_loglin_out],
         ['orig', 'interp', 'interp_log', 'interp_loglog', 'interp_loglin'],
         "Wavelength [m]",
         "scattering_cross_section [$m^{-2}$]",
         ['+', 'x', '1', '2', '3'],
         'log',
         'linear' )

plt_func(ax_sigma_scat_log, 
         [cloud_wavelength,opac_bin_centers_wavelengths,opac_bin_centers_wavelengths,opac_bin_centers_wavelengths,opac_bin_centers_wavelengths,opac_bin_centers_wavelengths],
         [cloud_scattering, cloud_scattering_out, cloud_scattering_log_out, cloud_scattering_loglog_out, cloud_scattering_loglin_out],
         ['orig', 'interp', 'interp_log', 'interp_loglog', 'interp_loglin'],
         "Wavelength [m]",
         "scattering_cross_section [$m^{-2}$]",
         ['+', 'x', '1', '2', '3'],
         'log',
         'log' )



### asymmetry

In [ ]:
fig, (ax_sigma_abs_lin, ax_sigma_abs_log)  = plt.subplots(1, 2, figsize=(2*14,10))

plt_func(ax_sigma_abs_lin, 
         [cloud_wavelength,opac_bin_centers_wavelengths,opac_bin_centers_wavelengths,opac_bin_centers_wavelengths],
         [cloud_asymmetry, cloud_asymmetry_out, cloud_asymmetry_loglin_out, cloud_asymmetry_loglog_out],
         ['orig', 'interp', 'interp_loglin', 'interp_loglog'],
         "Wavelength [m]",
         "asymmetry parameter",
         ['+', 'x', '1', '2'],
         'log',
         'linear' )

plt_func(ax_sigma_abs_log, 
         [cloud_wavelength,opac_bin_centers_wavelengths,opac_bin_centers_wavelengths,opac_bin_centers_wavelengths],
         [cloud_asymmetry, cloud_asymmetry_out, cloud_asymmetry_loglin_out, cloud_asymmetry_loglog_out],
         ['orig', 'interp', 'interp_loglin', 'interp_loglog'],
         "Wavelength [m]",
         "asymmetry parameter",
         ['+', 'x', '1', '2'],
         'log',
         'log' )




pass

In [ ]:
with h5py.File(cloud_sample_output, "w") as f:
    lambda_dset = f.create_dataset("wavelength", (len(opac_bin_centers_wavelengths),), dtype=np.float64)
    absorption_dset = f.create_dataset("absorption", cloud_absorption_loglog_out.shape, dtype=np.float64)
    scattering_dset = f.create_dataset("scattering", cloud_scattering_loglog_out.shape, dtype=np.float64)
    asymmetry_dset = f.create_dataset("asymmetry", cloud_asymmetry_loglog_out.shape, dtype=np.float64)
    lambda_dset[...] = opac_bin_centers_wavelengths
    absorption_dset[...] = cloud_absorption_loglog_out
    scattering_dset[...] = cloud_scattering_loglog_out
    asymmetry_dset[...] = cloud_asymmetry_loglog_out
    
    